<a href="https://colab.research.google.com/github/ValentinaEmili/Ethnicity-recognition/blob/main/Dlib/68_landmarks/VGGFace_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface
!pip install tqdm

In [ ]:
%pip install gdown
import gdown

# URL for the folder
url = "https://drive.google.com/drive/folders/1ZKMUq6U6pRw7zcP6AkGxoQVX3LnMnGXC?usp=share_link"
gdown.download_folder(url, quiet=True)

In [4]:
# Set the working directory for DeepFace
import os
os.environ['DEEPFACE_HOME'] = '/content'

In [ ]:
import os
import cv2
import numpy as np
import shutil
import zipfile
import dlib
from tqdm import tqdm
from google.colab.patches import cv2_imshow
from deepface import DeepFace

In [ ]:
# documentation: https://github.com/serengil/deepface

def main():
  folder_path = "/content/splitted_dataset"
  for file_name in os.listdir(folder_path):

    # zipped files in the folder biometric_systems_dataset
    if file_name.endswith(".zip"):

      # full path to the zip file
      file_path = os.path.join(folder_path, file_name)

      # create folder in the same path with the name of the zip file
      extract_folder = os.path.join(folder_path, file_name.replace(".zip", ""))
      os.makedirs(extract_folder, exist_ok=True)

      # open and extract the file
      with zipfile.ZipFile(file_path, mode="r") as zip_ref:
        zip_ref.extractall(extract_folder)

  feature_folders = ["eyes", "nose", "mouth"]
  sets = ["train", "test"]
  for feature in feature_folders:
    feature_folder_path = os.path.join(folder_path, feature)
    os.makedirs(feature, exist_ok=True)

    #remove the extracted zip files
    if feature_folder_path.endswith(".zip"):
      os.remove(feature_folder_path)
    for s in sets:

      # output folder for the embeddings for each feature
      os.makedirs(os.path.join(feature, s), exist_ok=True)
      set_folder_path = os.path.join(feature_folder_path, s)
      for file_name in tqdm(os.listdir(set_folder_path), desc="Processing Images"):
        if file_name.endswith(".jpg"):
          image_path = os.path.join(set_folder_path, file_name)
          embedding = DeepFace.represent(img_path=image_path, model_name="VGG-Face", enforce_detection=False)
          feature_vector = np.array(embedding[0]['embedding'])
          np.save(os.path.join(os.path.join(feature, s), file_name.replace(".jpg", ".npy")), feature_vector)
    shutil.make_archive(feature, "zip", feature)

if __name__ == "__main__":
    main()